In [35]:
from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")


In [27]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.schema import Document
import re

# Load PDF and remove a known unwanted boilerplate sentence on load
loader = PyPDFLoader("INTENSIVE GRAMMAR.pdf")
docs = loader.load()

# Exact sentence to remove (leading newline optional). Case-insensitive match.
target = "Mọi thng tin về khoá học vui lòng liên hệ Zalo official: The Forum Education – có tick xanh tại ô search"
pattern = re.compile(r"\n?" + re.escape(target), flags=re.IGNORECASE)

cleaned_docs = []
for d in docs:
    text = getattr(d, 'page_content', str(d))
    # remove the unwanted sentence if present
    new_text = pattern.sub('', text)
    # normalize excessive blank lines
    new_text = re.sub(r"\n{3,}", "\n\n", new_text)
    if new_text != text:
        meta = d.metadata if hasattr(d, 'metadata') else {}
        cleaned_docs.append(Document(page_content=new_text, metadata=meta))
    else:
        cleaned_docs.append(d)

docs = cleaned_docs
print(f"Loaded {len(docs)} pages; removed unwanted boilerplate where found.")

Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 55 0 (offset 0)
Ignoring wrong pointing object 92 0 (offset 0)
Ignoring wrong pointing object 137 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 55 0 (offset 0)
Ignoring wrong pointing object 92 0 (offset 0)
Ignoring wrong pointing object 137 0 (offset 0)


Loaded 41 pages; removed unwanted boilerplate where found.


In [28]:
# Inspect the loaded documents from the PDF
print("Total docs loaded:", len(docs))
print(docs[0].page_content[:1000])
print(docs[1].page_content[:1000])


Total docs loaded: 41
       1  
PHẦN 1: ĐỘNG TỪ VÀ THÌ (VERB TENSES) 1.1. Tổng quan về thì động từ trong tiếng Anh Thì động từ trong tiếng Anh thể hiện thời gian và trạng thái của hành động hoặc trạng thái. Có 12 thì cơ bản trong tiếng Anh, chia thành ba thời chính: hiện tại, quá khứ và tương lai, mỗi thời có bốn thì nhỏ (đơn, tiếp diễn, hoàn thành, hoàn thành tiếp diễn). 1.1.1. Thì hiện tại (Present Tenses) • Hiện tại đơn (Present Simple): Diễn tả thói quen, sự thật hiển nhiên, lịch trình. o Cấu trúc:  § Khẳng định: S + V(s/es) § Phủ định: S + do/does + not + V-nguyên thể § Nghi vấn: Do/Does + S + V-nguyên thể? o Ví dụ:  § She works every day. § They do not play football. § Does he like coffee? • Hiện tại tiếp diễn (Present Continuous): Diễn tả hành động đang diễn ra tại thời điểm nói hoặc kế hoạch tương lai gần. o Cấu trúc:  § S + am/is/are + V-ing o Ví dụ:  § I am studying now. § They are meeting us tomorrow. 
       2  
• Hiện tại hoàn thành (Present Perfect): Diễn tả hành động đã

In [29]:
# Imports for text chunking
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List
from langchain.schema import Document

In [30]:
# --- Chunk the loaded PDF documents and test examples ---
print('Docs loaded:', len(docs))
# Use RecursiveCharacterTextSplitter to create chunks sized for embeddings/LLMs
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(docs)
print('Total chunks produced:', len(chunks))

# Show a few sample chunks (preview)
for i, c in enumerate(chunks[:5]):
    src = c.metadata.get('source') if isinstance(c.metadata, dict) else None
    print(f'\n--- Chunk {i} (source={src}) ---')
    print(c.page_content[:500].replace('\n', ' '))

# Basic statistics about chunk sizes (word counts)
lengths = [len(c.page_content.split()) for c in chunks]
import statistics
if lengths:
    print('Chunk words: min', min(lengths), 'median', statistics.median(lengths), 'max', max(lengths))

# Save chunks to a JSONL file for later indexing (UTF-8)
import json
out_path = 'INTENSIVE_GRAMMAR_chunks.jsonl'
with open(out_path, 'w', encoding='utf-8') as f:
    for c in chunks:
        meta = c.metadata if hasattr(c, 'metadata') else {}
        obj = {'text': c.page_content, 'metadata': meta}
        f.write(json.dumps(obj, ensure_ascii=False) + '\n')
print('Saved chunks to', out_path)

# Quick test: load the saved JSONL and print first item to verify
with open(out_path, 'r', encoding='utf-8') as f:
    first = f.readline()
    print('First saved chunk preview:', first[:500])

Docs loaded: 41
Total chunks produced: 54

--- Chunk 0 (source=INTENSIVE GRAMMAR.pdf) ---
1   PHẦN 1: ĐỘNG TỪ VÀ THÌ (VERB TENSES) 1.1. Tổng quan về thì động từ trong tiếng Anh Thì động từ trong tiếng Anh thể hiện thời gian và trạng thái của hành động hoặc trạng thái. Có 12 thì cơ bản trong tiếng Anh, chia thành ba thời chính: hiện tại, quá khứ và tương lai, mỗi thời có bốn thì nhỏ (đơn, tiếp diễn, hoàn thành, hoàn thành tiếp diễn). 1.1.1. Thì hiện tại (Present Tenses) • Hiện tại đơn (Present Simple): Diễn tả thói quen, sự thật hiển nhiên, lịch trình. o Cấu trúc:  § Khẳng định: S + V

--- Chunk 1 (source=INTENSIVE GRAMMAR.pdf) ---
2   • Hiện tại hoàn thành (Present Perfect): Diễn tả hành động đã xảy ra và có kết quả đến hiện tại hoặc trải nghiệm. o Cấu trúc:  § S + have/has + V3/V-ed o Ví dụ:  § She has visited London twice. § We have finished our work. • Hiện tại hoàn thành tiếp diễn (Present Perfect Continuous): Diễn tả hành động bắt đầu trong quá khứ và vẫn tiếp tục ở hiện tại. o Cấ

In [31]:
# --- Create embeddings and FAISS vector store ---
print("⏳ Creating embeddings for chunks (this may take a minute)...")

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Use a lightweight local embedding model (no API calls)
# all-MiniLM-L6-v2: 384-dim embeddings, fast and good quality
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create FAISS vector store from chunks
vector_store = FAISS.from_documents(chunks, embedding_model)
print(f"✅ Vector store created with {len(chunks)} chunks")

# Save the vector store to disk for reuse
vector_store_path = "INTENSIVE_GRAMMAR_faiss_index"
vector_store.save_local(vector_store_path)
print(f"✅ Saved vector store to {vector_store_path}/")

# --- Test similarity search ---
print("\n" + "="*50)
print("Testing similarity search...")
print("="*50)

# Example queries to test
test_queries = [
    "Thì quá khứ tiếp diễn",

]

for query in test_queries:
    print(f"\n🔍 Query: '{query}'")
    # search: returns top-k most similar chunks
    results = vector_store.similarity_search(query, k=5)
    for i, result in enumerate(results, 1):
        content_preview = result.page_content[:300].replace('\n', ' ')
        meta = result.metadata if hasattr(result, 'metadata') else {}
        print(f"  Result {i} (page={meta.get('page')}):")
        print(f"    {content_preview}...")

# --- Test similarity_search_with_scores ---
print("\n" + "="*50)
print("Search with scores (lower is better)...")
print("="*50)

query = "grammar rules"
results_with_scores = vector_store.similarity_search_with_score(query, k=5)
for i, (doc, score) in enumerate(results_with_scores, 1):
    content_preview = doc.page_content[:200].replace('\n', ' ')
    print(f"  Result {i} (score={score:.4f}):")
    print(f"    {content_preview}...")

print("\n✅ FAISS vector store is ready for RAG!")
print(f"   - {len(chunks)} chunks indexed")
print(f"   - Saved to: {vector_store_path}/")
print("   - Use vector_store.similarity_search(query, k=5) to retrieve top-k chunks")

⏳ Creating embeddings for chunks (this may take a minute)...
✅ Vector store created with 54 chunks
✅ Saved vector store to INTENSIVE_GRAMMAR_faiss_index/

Testing similarity search...

🔍 Query: 'Thì quá khứ hoàn thành tiếp diễn'
  Result 1 (page=7):
    1.2.5. Quá khứ tiếp diễn (Past Continuous) Lỗi 1: Không sử dụng quá khứ tiếp diễn đúng ngữ cảnh Sai: When I arrived, they played football. Sửa: When I arrived, they were playing football. Giải thích: Hành động đang diễn ra trong quá khứ ("they were playing") khi một hành động khác xảy đến ("I arrive...
  Result 2 (page=7):
    hoàn thành khi không cần thiết Sai: She had gone to the store yesterday. Sửa: She went to the store yesterday. Giải thích: Nếu chỉ có một hành động trong quá khứ, và không có hành động nào khác để so sánh thời gian, thì dùng quá khứ đơn. 1.2.7. Tương lai đơn (Future Simple) Lỗi 1: Sử dụng "will" cho...
  Result 3 (page=6):
    7   Sửa: They have eaten lunch. Giải thích: Động từ phân từ "eat" là "eaten". Cần chú ý 

In [33]:
# --- Create RAG chain with OpenAI and custom prompt ---
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough

if not openai_api_key:
    print("⚠️ OPENAI_API_KEY not found in environment. Please set it before running this cell.")
else:
    # Initialize OpenAI LLM
    llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0.3,
        api_key=openai_api_key
    )
    
    # Custom prompt template for the grammar teacher role
    prompt_template = PromptTemplate(
        input_variables=["context", "question"],
        template=""""You are an English grammar teacher. "
            "A Vietnamese student has asked you a question about grammar.\n\n"
            "RETRIEVED CONTEXT:\n{context}\n\n"
            "STUDENT QUESTION:\n{question}\n\n"
            "Please answer using Vietnamese language following these steps:\n"
            "1. Carefully read the CONTEXT retrieved from the database. Only use information that appears in the CONTEXT."
            "2. Give a short and clear explanation of the grammar point the student is asking about. Explain the meaning, Explain the usage, Explain the structure (if included in the context).\n"
            "3. Provide an example (use examples from the context if available). If the retrieved context contains examples, include at least one example verbatim and label it exactly as 'Ví dụ:'.\n"
            "4. Re-explain the concept using simpler Vietnamese so that a language learner can understand it easily.\n"
            "5. If the concept does not exist in the retrieved context, tell me honestly.\n\n"
            "Your response:"
"""
    )
    
    # Create a retrieval function that gets context from vector store
    def get_context(query, k=5):
        """Retrieve top-k chunks from FAISS and format as context.
        If none of the top-k contain example markers (e.g. 'ví dụ', 'đáp án'),
        try a lightweight fallback by scanning the saved JSONL chunks for a chunk
        containing example markers and append it to the context.
        """
        results = vector_store.similarity_search(query, k=5)
        texts = [doc.page_content for doc in results]

        # Check for example markers in retrieved chunks
        markers = ['ví dụ', 'đáp án', 'ví-dụ', 'vídu', 'example', 'ans:']
        def has_example(text):
            t = text.lower()
            return any(m in t for m in markers)

        contains_example = any(has_example(t) for t in texts)

        # Fallback: scan saved JSONL for a chunk with an example marker and append it
        if not contains_example:
            try:
                import json
                with open('INTENSIVE_GRAMMAR_chunks.jsonl', 'r', encoding='utf-8') as f:
                    for ln in f:
                        obj = json.loads(ln)
                        txt = obj.get('text', '').lower()
                        if any(m in txt for m in markers):
                            # append this chunk's original text to the context and stop
                            texts.append(obj.get('text', ''))
                            contains_example = True
                            break
            except FileNotFoundError:
                # Saved JSONL not found; skip fallback
                pass

        context = "\n\n---\n\n".join(texts)
        return context
    
    # Strengthen prompt: require verbatim example labeled 'Ví dụ:' if available
    prompt_template = PromptTemplate(
        input_variables=["context", "question"],
        template=(
            "You are an English grammar teacher. "
            "A Vietnamese student has asked you a question about grammar.\n\n"
            "RETRIEVED CONTEXT:\n{context}\n\n"
            "STUDENT QUESTION:\n{question}\n\n"
            "Please answer using Vietnamese language following these steps:\n"
            "1. Carefully read the CONTEXT retrieved from the database. Only use information that appears in the CONTEXT."
            "2. Give a short and clear explanation of the grammar point the student is asking about. Explain the meaning, Explain the usage, Explain the structure (if included in the context).\n"
            "3. Provide an example (use examples from the context if available). If the retrieved context contains examples, include at least one example verbatim and label it exactly as 'Ví dụ:'.\n"
            "4. Re-explain the concept using simpler Vietnamese so that a language learner can understand it easily.\n"
            "5. If the concept does not exist in the retrieved context, tell me honestly.\n\n"
            "Your response:"
        )
    )

    # Build RAG chain using LangChain's pipe operator
    rag_chain = (
        {
            "context": lambda x: get_context(x["question"]),
            "question": RunnablePassthrough()
        }
        | prompt_template
        | llm
    )

    # Diagnostic helper to print top-k results and whether they contain example markers
    def diagnose_question(question, k=5):
        print(f"\n[DIAGNOSTIC] Running similarity_search_with_score for: '{question}' (k={k})")
        try:
            results_with_scores = vector_store.similarity_search_with_score(question, k=k)
        except Exception as e:
            print("Error running similarity_search_with_score:", e)
            return

        markers = ['ví dụ', 'đáp án', 'ví-dụ', 'vídu', 'example', 'ans:']
        for i, (doc, score) in enumerate(results_with_scores, start=1):
            text = getattr(doc, 'page_content', str(doc))
            preview = text[:800].replace('\n', ' ')
            lowered = text.lower()
            contains_example = any(m in lowered for m in markers)
            print(f"--- Result {i} (score={score:.4f}) contains_example={contains_example} ---")
            print("metadata:", getattr(doc, 'metadata', {}))
            print(preview)
            print('\n')

        # Quick JSONL search for a common tense phrase to show where examples live
        try:
            import json
            pattern = 'hiện tại hoàn thành'
            found = 0
            with open('INTENSIVE_GRAMMAR_chunks.jsonl', 'r', encoding='utf-8') as f:
                for ln in f:
                    obj = json.loads(ln)
                    txt = obj.get('text', '').lower()
                    if pattern in txt:
                        found += 1
                        print('\n--- Found chunk containing pattern (preview) ---')
                        print(obj.get('text', '')[:1200].replace('\n', ' '))
                        break
            if found == 0:
                print(f"No saved chunk containing '{pattern}' found in INTENSIVE_GRAMMAR_chunks.jsonl")
        except FileNotFoundError:
            print('INTENSIVE_GRAMMAR_chunks.jsonl not found; run the chunking cell first')
    
    print("✅ RAG chain initialized!")
    print("\n" + "="*70)
    print("INTERACTIVE GRAMMAR TEACHER - Testing with Examples")
    print("="*70)
    
    # Test queries to demonstrate the RAG chain
    test_questions = [
        "sử dụng thì hiện tại hoàn thành như thế nào?",
        "thì tương lai đơn là gì ? "
    ]
    
    for i, question in enumerate(test_questions, 1):
        print(f"\n{'='*70}")
        print(f"❓ Student Question {i}: {question}")
        print(f"{'='*70}")
        
        try:
            # Retrieve context
            context = get_context(question)
            print(f"\n📚 Retrieved Context (first 500 chars):")
            print(context[:500] + "..." if len(context) > 500 else context)
            
            # Get response from RAG chain
            print(f"\n🎓 Teacher Response:")
            print("-" * 70)
            # Pass a mapping matching the chain's input contract: {'question': ...}
            response = rag_chain.invoke({"question": question})
            # response may be a LangChain message object, a dict, or other runnable result.
            # Extract text content safely.
            if hasattr(response, 'content'):
                answer_text = response.content
            elif isinstance(response, dict) and 'content' in response:
                answer_text = response['content']
            else:
                answer_text = str(response)
            print(answer_text)
            print("-" * 70)
            
        except Exception as e:
            print(f"❌ Error processing question: {e}")
    
    print("\n" + "="*70)
    print("✅ RAG Grammar Teacher is ready!")
    print("="*70)


✅ RAG chain initialized!

INTERACTIVE GRAMMAR TEACHER - Testing with Examples

❓ Student Question 1: Động từ là gì?

📚 Retrieved Context (first 500 chars):
38  
Giải thích: Tiền tố phủ định của "responsible" là "ir-". Lỗi 3: Nhầm lẫn giữa từ gốc và từ biến thể Sai: They made a great improve. Sửa: They made a great improvement. Giải thích: "Improvement" là danh từ, "improve" là động từ.  4.4. Chiến lược nhận diện và sửa lỗi 4.4.1. Xác định chức năng từ trong câu • Hỏi xem từ đó đang làm gì trong câu: làm chủ ngữ, động từ, tính từ, trạng từ... • Dựa vào vị trí của từ để xác định loại từ cần thiết. 4.4.2. Nhận diện hậu tố và tiền tố • Hậu tố thường ch...

🎓 Teacher Response:
----------------------------------------------------------------------
1. Động từ là từ dùng để diễn tả hành động hoặc trạng thái. Trong tiếng Anh, động từ có vai trò rất quan trọng trong câu, giúp người nghe hiểu được hành động mà chủ ngữ đang thực hiện.

2. Nói đơn giản, động từ là những từ chỉ hành động mà người ho

In [34]:
# === DIAGNOSTIC: Investigate why Q3 (Quá khứ hoàn thành tiếp diễn là gì?) lacks examples ===
print("\n" + "="*70)
print("DIAGNOSTIC: Investigating Q3 Missing Examples")
print("="*70)

q3 = "Quá khứ hoàn thành tiếp diễn là gì ?"
print(f"\nQuestion: {q3}")

# Test 1: Check what top-k results are retrieved with scores
print("\n[Step 1] Retrieving top-10 chunks with scores...")
try:
    results_scored = vector_store.similarity_search_with_score(q3, k=5)
    for i, (doc, score) in enumerate(results_scored, 1):
        text = doc.page_content[:500].replace('\n', ' ')
        has_example = any(m in doc.page_content.lower() for m in ['ví dụ', 'đáp án', 'example'])
        print(f"  [{i}] score={score:.4f} | has_example={has_example}")
        print(f"      {text}...")
        print()
except Exception as e:
    print(f"Error retrieving: {e}")

# Test 2: Search the saved JSONL for chunks about 'Quá khứ hoàn thành tiếp diễn'
print("\n[Step 2] Searching saved JSONL for content about 'quá khứ hoàn thành tiếp diễn'...")
try:
    import json
    pattern = 'quá khứ hoàn thành tiếp diễn'
    found_chunks = []
    with open('INTENSIVE_GRAMMAR_chunks.jsonl', 'r', encoding='utf-8') as f:
        for ln in f:
            obj = json.loads(ln)
            txt = obj.get('text', '').lower()
            if pattern in txt:
                found_chunks.append(obj.get('text', ''))
    print(f"  Found {len(found_chunks)} chunk(s) in JSONL containing the pattern.")
    if found_chunks:
        for idx, chunk in enumerate(found_chunks[:3], 1):  # show first 3
            has_ex = any(m in chunk.lower() for m in ['ví dụ', 'đáp án', 'example'])
            print(f"\n  --- JSONL Chunk {idx} (has examples: {has_ex}) ---")
            print(f"  {chunk[:800].replace(chr(10), ' ')}")
except Exception as e:
    print(f"Error searching JSONL: {e}")

# Test 3: Run get_context for Q3 and show what context is constructed
print("\n[Step 3] Context constructed by get_context() for Q3...")
try:
    context = get_context(q3, k=5)
    print(f"  Total context length: {len(context)} chars")
    print(f"  First 1000 chars of context:")
    print(f"  {context[:1000].replace(chr(10), ' ')}")
except Exception as e:
    print(f"Error getting context: {e}")

# Test 4: Summary
print("\n[Summary]")
print("  If top-k results don't contain example markers, the fallback should scan JSONL.")
print("  If examples exist in JSONL but aren't retrieved, try:")
print("    - Increase k (currently 5) to retrieve more chunks.")
print("    - Use a different semantic query or embedding model.")
print("    - Manually inspect the PDF to confirm examples exist and aren't corrupted.")
print("="*70)


DIAGNOSTIC: Investigating Q3 Missing Examples

Question: Quá khứ hoàn thành tiếp diễn là gì ?

[Step 1] Retrieving top-10 chunks with scores...
  [1] score=0.9190 | has_example=False
      hoàn thành khi không cần thiết Sai: She had gone to the store yesterday. Sửa: She went to the store yesterday. Giải thích: Nếu chỉ có một hành động trong quá khứ, và không có hành động nào khác để so sánh thời gian, thì dùng quá khứ đơn. 1.2.7. Tương lai đơn (Future Simple) Lỗi 1: Sử dụng "will" cho kế hoạch đã định trước...

  [2] score=0.9296 | has_example=False
      1.2.5. Quá khứ tiếp diễn (Past Continuous) Lỗi 1: Không sử dụng quá khứ tiếp diễn đúng ngữ cảnh Sai: When I arrived, they played football. Sửa: When I arrived, they were playing football. Giải thích: Hành động đang diễn ra trong quá khứ ("they were playing") khi một hành động khác xảy đến ("I arrived"). Lỗi 2: Sử dụng quá khứ tiếp diễn cho hành động ngắn, hoàn tất Sai: He was slipping and was falling. Sửa: He slipped and fell. Giải t